In [ ]:
"""This code helps to find out feature multipliers for KNN.
This is shown using some features derived by me but this method can be extended for other features as well.
One needs to derive his own features and then apply similar approach to get the correct weights.
This code has been discussed here:
https://www.kaggle.com/chopra/facebook-v-predicting-check-ins/logistic-regression-to-find-knn-weights/code
"""

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from sklearn.linear_model import LogisticRegression
lr = LogisticRegression()

train = pd.read_csv("../engineered_train.csv")

#select a single x_y_grid at random
train = train[(train["x"]>4500) &(train["x"]<5000) &(train["y"]>2000) &(train["y"]<2300)]

print("train created")

In [ ]:
test = train.sample(axis = 0,frac = 0.05)
print ("selected_part and test created")
features = ["x","y","hour_of_day","day_of_week","month_of_year","accuracy"]
fw = [1,1,0,0,0,0] #at first iteration
fw = [ 1.  ,      1.43696871,  1.26585057,  0.78156837,  1.28182383,  7.01014412] #at second iteration

print (len(test))

In [ ]:
colname = str(features)
test[colname] = list
index = iter(test.index)
test["done"] = 0
count = 0
for i in index:
    new_ld = pd.DataFrame(columns = features)
    for j in range(15):
        new_ld1 = abs(train[features] - test.loc[i][features])
        new_ld1 = new_ld1.drop(i)
        new_ld1["target"] = (train["place_id"] != test.loc[i]["place_id"]) + 0
        new_ld1["x+y"] = np.sum(new_ld1[features]*fw,axis = 1)#(new_ld1["x"])+(new_ld1["y"])#
        new_ld1 = new_ld1.sort("x+y")[0:50]
        count += 1
        try:
            i = next(index)
        except:
            break
        true = new_ld1[new_ld1["target"] == 0]
        false = new_ld1[new_ld1["target"] != 0]
        if (len(true)< 10) | (len(false)< 10):
            continue
        new_ld = new_ld.append(new_ld1)
    lr.fit(new_ld[features],new_ld["target"])
    test.set_value(i,colname,lr.coef_.ravel())
    test.set_value(i,"done",1)
    print ("current status: sample number",count)

In [ ]:
#average or sum all the multipliers to get overall multiplier
actual_test2 = test[test["done"]==1]
final_weights = np.array([0,0,0,0,0,0])
for lists in actual_test2[colname]:
    final_weights = final_weights + lists


print (features) 
print ("corresponding weights")
print (final_weights/final_weights[0])